In [4]:
import pandas as pd
import numpy as np

from route import Route
# from weather_forecast import WeatherForecast
# from api_data import ApiData
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from FUNCTIONS import get_current_location

In [ ]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [ ]:
current_position = {'longitude': 130.868566,
                    'latitude': -12.432466}
delta_spacing = 100000.0 # in meters
number_sites = 3
route = Route()
# route.get_final_data(current_position)
# route.get_final_data(current_position, delta_spacing=delta_spacing)
dataframe = route.get_route_data(current_position, number_sites=number_sites)

In [5]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved. 
   name site_id  longitude  latitude altitude
0    0  584987    130.869  -12.4325       18
1    1  584988    133.804  -23.8618      557
2    2  584989    138.609  -34.9352       52


In [18]:
# api_executer.add_sites(dataframe, print_is_requested=False)
api_executer.delete_all_sites()

All sites have been deleted: 
 Empty DataFrame
Columns: [name, site_id, longitude, latitude, altitude]
Index: []


In [ ]:
position = {
    'latitude': -13.0,
    'longitude': 134.0
}
api_requester.get_site_edit(584978, True, name="aa")
api_requester.get_site_edit(584979, True, position=position)
api_requester.get_site_edit(584980, True, name="df", position=position)

In [6]:
forecasts = api_requester.get_solar_forecast()
# forecasts = api_requester.get_solar_forecast_cloudmove()
print(forecasts)

Response status from getforecast: OK.
Solar forecast have been retrieved.
                              gk  gh_max  gh         tt    ff        rr  \
site_id time                                                              
584987  2023-08-25 13:00:00  0.0     0.0   0  23.100000   7.0  0.000000   
        2023-08-25 13:10:00  0.0     0.0   0  22.916667   7.0  0.000000   
        2023-08-25 13:20:00  0.0     0.0   0  22.733333   7.0  0.000000   
        2023-08-25 13:30:00  0.0     0.0   0  22.550000   7.0  0.000000   
        2023-08-25 13:40:00  0.0     0.0   0  22.366667   7.0  0.000000   
...                          ...     ...  ..        ...   ...       ...   
584989  2023-09-03 11:20:00  0.0     0.0   0  13.133333  13.0  0.005556   
        2023-09-03 11:30:00  0.0     0.0   0  13.100000  13.0  0.008333   
        2023-09-03 11:40:00  0.0     0.0   0  13.066667  13.0  0.011111   
        2023-09-03 11:50:00  0.0     0.0   0  13.033333  13.0  0.013889   
        2023-09-03 12:00:0

In [ ]:
index = forecasts.index

# Access the values of time and site_id
time_values = index.get_level_values('time')
site_id_values = index.get_level_values('site_id')

# Print the values
print("Time values:", time_values)
print("Site ID values:", site_id_values)

In [16]:
# Extract all 'rr' values for all site_id and time combinations
# rr_values = forecasts.loc[:, ('rr', slice(None), slice(None))].droplevel('variable')
rr_values = forecasts.loc[:, ('rh')]
rr_values = rr_values.to_frame()
print(rr_values)
print(rr_values.max(), rr_values.min())
# # Assuming your MultiIndex DataFrame is named "forecast_data"
# pivot_table = rr_values.pivot(index='time', columns='site_id', values='tt')
# print(pivot_table)
rr_values.to_csv('rh_table.csv')

# print(rr_values)

                                    rh
site_id time                          
584987  2023-08-25 13:00:00  71.000000
        2023-08-25 13:10:00  71.500000
        2023-08-25 13:20:00  72.000000
        2023-08-25 13:30:00  72.500000
        2023-08-25 13:40:00  73.000000
...                                ...
584989  2023-09-03 11:20:00  68.333333
        2023-09-03 11:30:00  68.500000
        2023-09-03 11:40:00  68.666667
        2023-09-03 11:50:00  68.833333
        2023-09-03 12:00:00  69.000000

[3873 rows x 1 columns]
rh    99.0
dtype: float64 rh    7.0
dtype: float64


In [2]:
import atmos
density = atmos.calculate('rho', Tv=300.0, p=990.0, p_units='hPa')
print(density)

AttributeError: module 'inspect' has no attribute 'getargspec'